smart grid simulation

----

In [2]:
from __future__ import annotations

from dataclasses import dataclass, field
from typing import Dict, Optional, List, Tuple, Literal
from enum import Enum, auto
from collections import deque
import heapq
import numpy as np
import pandas as pd
import warnings

# silence future pandas groupby warning in our summaries (optional)
warnings.filterwarnings("ignore", category=FutureWarning, module="pandas.core.groupby")

# reproducibility
DEFAULT_SEED = 42
rng = np.random.default_rng(DEFAULT_SEED)

Time = float
ID = int


In [3]:
class PolicyType(Enum):
    FIFO = auto()
    NPPS = auto()   # Non-Preemptive Priority Scheduling
    WRR  = auto()   # Weighted Round Robin
    EDF  = auto()   # Earliest Deadline First

@dataclass
class SimConfig:
    # --- non-defaults first ---
    chi_arrival: float                   # χ — Poisson arrival rate
    lambda_ctrl: float                   # λ1 — controller service rate (exp)
    lambda_res: Dict[str, float]         # λ2 per resource {'PV':6, 'BAT':12, ...}
    setup_delay: float                   # t — fixed setup delay (ctrl & res)
    T: Time                              # total horizon
    N_ctrl: int                          # number of controller servers (M/M/N)
    transfer_overhead: Dict[str, float]  # C_time per resource (routing delay)
    route_probs: Dict[str, float]        # P routing probabilities to resources

    # --- defaults below ---
    ctrl_policy: PolicyType = PolicyType.FIFO

    # priorities & default deadlines (slots interpreted in same time unit)
    group_priority: Dict[str, int] = field(
        default_factory=lambda: {"essential": 3, "delay_sensitive": 2, "delay_tolerant": 1}
    )
    default_deadline_slots: Dict[str, Tuple[int,int]] = field(
        default_factory=lambda: {"delay_sensitive": (1,4), "delay_tolerant": (3,12)}
    )
    seed: int = DEFAULT_SEED

    def validate(self):
        assert self.chi_arrival > 0, "chi_arrival must be > 0"
        assert self.lambda_ctrl > 0, "lambda_ctrl must be > 0"
        assert all(v > 0 for v in self.lambda_res.values()), "All lambda_res must be > 0"
        assert self.N_ctrl >= 1, "N_ctrl must be >= 1"
        assert self.T > 0, "T must be > 0"
        s = sum(self.route_probs.values())
        assert abs(s - 1.0) < 1e-8, f"route_probs must sum to 1 (got {s})"
        # keys must align
        for r in self.transfer_overhead:
            assert r in self.lambda_res, f"transfer_overhead key {r} missing in lambda_res"
        for r in self.lambda_res:
            assert r in self.transfer_overhead, f"lambda_res key {r} missing in transfer_overhead"


In [4]:
from math import inf

@dataclass
class Request:
    rid: ID
    group: str                          # 'essential' | 'delay_sensitive' | 'delay_tolerant' | ...
    priority: int                       # for NPPS
    arrival_time: Time
    deadline_time: Optional[Time] = None

    # timestamps through system
    t_ctrl_start: Optional[Time] = None
    t_ctrl_end:   Optional[Time] = None
    routed_to:    Optional[str]  = None
    t_route_done: Optional[Time] = None
    t_res_start:  Optional[Time] = None
    t_res_end:    Optional[Time] = None

class BaseQueuePolicy:
    def push(self, req: Request) -> None: ...
    def pop(self, now: Time) -> Optional[Request]: ...
    def __len__(self) -> int: ...

class FIFOQueue(BaseQueuePolicy):
    def __init__(self): self._q = deque()
    def push(self, req: Request) -> None: self._q.append(req)
    def pop(self, now: Time) -> Optional[Request]:
        return self._q.popleft() if self._q else None
    def __len__(self) -> int: return len(self._q)

class NPPSQueue(BaseQueuePolicy):
    """Higher priority first; FIFO within same priority."""
    def __init__(self):
        self._counter = 0
        self._heap: List[Tuple[int, int, Request]] = []  # (-priority, order, req)
    def push(self, req: Request) -> None:
        self._counter += 1
        heapq.heappush(self._heap, (-int(req.priority), self._counter, req))
    def pop(self, now: Time) -> Optional[Request]:
        if not self._heap: return None
        _, _, req = heapq.heappop(self._heap)
        return req
    def __len__(self) -> int: return len(self._heap)

class EDFQueue(BaseQueuePolicy):
    """Earliest absolute deadline first; FIFO on ties; NaN/None treated as +inf."""
    def __init__(self):
        self._counter = 0
        self._heap: List[Tuple[float, int, Request]] = []  # (deadline, order, req)
    def push(self, req: Request) -> None:
        self._counter += 1
        dl = req.deadline_time if (req.deadline_time is not None) else inf
        heapq.heappush(self._heap, (float(dl), self._counter, req))
    def pop(self, now: Time) -> Optional[Request]:
        if not self._heap: return None
        _, _, req = heapq.heappop(self._heap)
        return req
    def __len__(self) -> int: return len(self._heap)

class WRRQueue(BaseQueuePolicy):
    """
    Weighted Round Robin over groups (each group has FIFO).
    Weights are positive numbers; internally normalized to small integers.
    """
    def __init__(self, group_weights: Dict[str, float]):
        assert all(w > 0 for w in group_weights.values()), "WRR weights must be > 0"
        self.fifos: Dict[str, deque] = {g: deque() for g in group_weights}
        # normalize to small integers for quotas
        g, w = zip(*group_weights.items())
        base = min(w)
        ints = {gi: max(1, int(round(wi / base))) for gi, wi in group_weights.items()}
        self.weights = ints
        self.groups = list(self.fifos.keys())
        self.idx = 0
        self.quota_left = dict(self.weights)

    def push(self, req: Request) -> None:
        if req.group not in self.fifos:
            # unseen group gets weight 1 lazily
            self.fifos[req.group] = deque()
            self.weights[req.group] = 1
            self.groups.append(req.group)
            self.quota_left[req.group] = 1
        self.fifos[req.group].append(req)

    def _advance(self): self.idx = (self.idx + 1) % len(self.groups)
    def _has_any(self) -> bool: return any(self.fifos[g] for g in self.groups)

    def pop(self, now: Time) -> Optional[Request]:
        if not self._has_any(): return None
        tried = 0
        while tried < len(self.groups):
            g = self.groups[self.idx]
            # reset cycle quotas if all exhausted
            if all(self.quota_left[x] == 0 for x in self.groups):
                self.quota_left = dict(self.weights)
            if self.quota_left[g] == 0 or not self.fifos[g]:
                self._advance(); tried += 1; continue
            self.quota_left[g] -= 1
            return self.fifos[g].popleft()
        # queues changed; reset quotas and retry once
        self.quota_left = dict(self.weights)
        return self.pop(now)


In [5]:
def make_queue(policy: PolicyType, *, wrr_weights: Optional[Dict[str, float]] = None) -> BaseQueuePolicy:
    if policy == PolicyType.FIFO: return FIFOQueue()
    if policy == PolicyType.NPPS: return NPPSQueue()
    if policy == PolicyType.EDF:  return EDFQueue()
    if policy == PolicyType.WRR:
        assert wrr_weights is not None and len(wrr_weights) > 0, "Provide wrr_weights for WRR"
        return WRRQueue(wrr_weights)
    raise NotImplementedError(policy)


In [6]:
class EventType(Enum):
    ARRIVAL = auto()       # to controller
    CTRL_FINISH = auto()   # controller finished a request
    RES_ARRIVAL = auto()   # arrival to a resource after transfer overhead
    RES_FINISH = auto()    # resource finished

@dataclass(order=True)
class Event:
    time: Time
    etype: EventType
    payload: dict = field(compare=False)

class ResourceServer:
    """Single-server M/M/1 resource with FIFO queue (per resource)."""
    def __init__(self, name: str, rate: float, setup_delay: float):
        self.name = name
        self.rate = rate
        self.setup_delay = setup_delay
        self.queue = deque()
        self.busy_until: Time = 0.0
        self.in_service: Optional[Request] = None

    def push(self, req: Request):
        self.queue.append(req)

    def try_start(self, now: Time, evq: List[Event]):
        if self.in_service is not None: return False
        if self.queue and self.busy_until <= now:
            req = self.queue.popleft()
            req.t_res_start = now
            svc = rng.exponential(1.0 / self.rate) + self.setup_delay
            finish = now + svc
            self.busy_until = finish
            self.in_service = req
            heapq.heappush(evq, Event(finish, EventType.RES_FINISH, {"res": self.name, "req": req}))
            return True
        return False

    def on_finish(self, now: Time):
        req = self.in_service
        if req: req.t_res_end = now
        self.in_service = None
        self.busy_until = now

class ResourcePool:
    def __init__(self, cfg: SimConfig):
        self.cfg = cfg
        self.resources: Dict[str, ResourceServer] = {
            rname: ResourceServer(rname, rate=lambda_rate, setup_delay=cfg.setup_delay)
            for rname, lambda_rate in cfg.lambda_res.items()
        }

    def route_choice(self) -> str:
        names = list(self.cfg.route_probs.keys())
        probs = np.array(list(self.cfg.route_probs.values()))
        return rng.choice(names, p=probs)

    def on_ctrl_finish(self, req: Request, now: Time, evq: List[Event]):
        rname = self.route_choice()
        req.routed_to = rname
        req.t_ctrl_end = now
        overhead = self.cfg.transfer_overhead.get(rname, 0.0)
        arrival_at_res = now + overhead
        req.t_route_done = arrival_at_res
        heapq.heappush(evq, Event(arrival_at_res, EventType.RES_ARRIVAL, {"res": rname, "req": req}))

    def on_res_arrival(self, rname: str, req: Request, now: Time, evq: List[Event]):
        res = self.resources[rname]
        res.push(req)
        res.try_start(now, evq)

    def on_res_finish(self, rname: str, now: Time, evq: List[Event]):
        res = self.resources[rname]
        res.on_finish(now)
        res.try_start(now, evq)

class MultiServerController:
    """Controller M/M/N with pluggable queue."""
    def __init__(self, cfg: SimConfig):
        self.cfg = cfg
        if cfg.ctrl_policy == PolicyType.WRR:
            # by default map weights from priorities (custom dict welcome too)
            wrr_weights = {g: float(p) for g, p in cfg.group_priority.items()}
            self.queue = make_queue(cfg.ctrl_policy, wrr_weights=wrr_weights)
        else:
            self.queue = make_queue(cfg.ctrl_policy)
        self.server_busy_until: List[Time] = [0.0] * cfg.N_ctrl
        self.in_service: Dict[int, Request] = {}

    def _first_free_server(self, now: Time) -> Optional[int]:
        for sid, busy_until in enumerate(self.server_busy_until):
            if busy_until <= now and self.in_service.get(sid) is None:
                return sid
        return None

    def try_start_service(self, now: Time, evq: List[Event]):
        started = 0
        while True:
            sid = self._first_free_server(now)
            if sid is None: break
            req = self.queue.pop(now)
            if req is None: break
            req.t_ctrl_start = now
            svc = rng.exponential(1.0 / self.cfg.lambda_ctrl) + self.cfg.setup_delay
            finish = now + svc
            self.server_busy_until[sid] = finish
            self.in_service[sid] = req
            heapq.heappush(evq, Event(finish, EventType.CTRL_FINISH, {"sid": sid, "req": req}))
            started += 1
        return started

    def on_arrival(self, req: Request, now: Time, evq: List[Event]):
        self.queue.push(req)
        self.try_start_service(now, evq)

    def on_finish(self, sid: int, now: Time) -> Request:
        self.server_busy_until[sid] = now
        return self.in_service.pop(sid)


In [7]:
def generate_next_arrival(prev_time: Time, chi: float) -> Time:
    return prev_time + rng.exponential(1.0 / chi)

def _sample_deadline_for_group(cfg: SimConfig, group: str, arrival_time: float) -> Optional[float]:
    if group in cfg.default_deadline_slots:
        lo, hi = cfg.default_deadline_slots[group]
        delta = float(rng.uniform(lo, hi))
        return arrival_time + delta
    return None

def bootstrap_requests(cfg: SimConfig, group_mix: Dict[str, float]) -> List[Request]:
    assert abs(sum(group_mix.values()) - 1.0) < 1e-8, "group_mix must sum to 1"
    t = 0.0
    rid = 0
    reqs: List[Request] = []
    groups = list(group_mix.keys())
    probs  = np.array(list(group_mix.values()))
    while t < cfg.T:
        t = generate_next_arrival(t, cfg.chi_arrival)
        if t >= cfg.T: break
        g = rng.choice(groups, p=probs)
        pr = cfg.group_priority.get(g, 1)
        dl = _sample_deadline_for_group(cfg, g, t)
        reqs.append(Request(
            rid=rid, group=g, priority=pr,
            arrival_time=t, deadline_time=dl
        ))
        rid += 1
    return reqs


In [8]:
def generate_next_arrival(prev_time: Time, chi: float) -> Time:
    return prev_time + rng.exponential(1.0 / chi)

def _sample_deadline_for_group(cfg: SimConfig, group: str, arrival_time: float) -> Optional[float]:
    if group in cfg.default_deadline_slots:
        lo, hi = cfg.default_deadline_slots[group]
        delta = float(rng.uniform(lo, hi))
        return arrival_time + delta
    return None

def bootstrap_requests(cfg: SimConfig, group_mix: Dict[str, float]) -> List[Request]:
    assert abs(sum(group_mix.values()) - 1.0) < 1e-8, "group_mix must sum to 1"
    t = 0.0
    rid = 0
    reqs: List[Request] = []
    groups = list(group_mix.keys())
    probs  = np.array(list(group_mix.values()))
    while t < cfg.T:
        t = generate_next_arrival(t, cfg.chi_arrival)
        if t >= cfg.T: break
        g = rng.choice(groups, p=probs)
        pr = cfg.group_priority.get(g, 1)
        dl = _sample_deadline_for_group(cfg, g, t)
        reqs.append(Request(
            rid=rid, group=g, priority=pr,
            arrival_time=t, deadline_time=dl
        ))
        rid += 1
    return reqs


In [9]:
def summarize_metrics(df: pd.DataFrame):
    """
    Returns (overall_df, per_group_df, miss_df or None)
    overall/per_group: count/valid, mean, median, p95 for core timing columns.
    miss_df: per-group deadline meeting/miss stats (+ __OVERALL__), if 'deadline' present.
    """
    if df.empty:
        return pd.DataFrame(), pd.DataFrame(), None

    cols = ["wait_ctrl", "svc_ctrl", "transfer", "wait_res", "svc_res", "total_response"]

    overall = df[cols].agg(["count", "mean", "median", lambda s: s.quantile(0.95)]).T
    overall.columns = ["count/valid", "mean", "median", "p95"]
    overall = overall.sort_index()

    # exclude grouping col from apply to avoid future behavior change
    per_group = df.groupby("group", dropna=False)[cols].apply(
        lambda g: g.agg(["count", "mean", "median", lambda s: s.quantile(0.95)]).T
    )
    per_group.columns = ["count/valid", "mean", "median", "p95"]

    # deadline miss table
    miss_table = None
    if "deadline" in df.columns:
        m = df.dropna(subset=["deadline"]).copy()
        if not m.empty:
            m["deadline_met"] = (m["res_end"] <= m["deadline"]).astype(int)
            g = m.groupby("group", dropna=False)["deadline_met"].agg(count="count", met="sum")
            g["misses"] = g["count"] - g["met"]
            g["miss_rate"] = g["misses"] / g["count"]
            overall_miss = pd.DataFrame({
                "count": [g["count"].sum()],
                "met": [g["met"].sum()],
                "misses": [g["misses"].sum()],
                "miss_rate": [g["misses"].sum() / max(1, g["count"].sum())]
            }, index=["__OVERALL__"])
            miss_table = pd.concat([g, overall_miss])

    return overall, per_group, miss_table


In [10]:
def run_simulation(cfg: SimConfig, group_mix: Dict[str, float]) -> pd.DataFrame:
    cfg.validate()
    # reset RNG per run
    global rng
    rng = np.random.default_rng(cfg.seed)

    reqs = bootstrap_requests(cfg, group_mix)
    controller = MultiServerController(cfg)
    pool = ResourcePool(cfg)

    # prime event queue with arrivals
    evq: List[Event] = []
    for req in reqs:
        heapq.heappush(evq, Event(req.arrival_time, EventType.ARRIVAL, {"req": req}))

    # main loop
    while evq:
        ev = heapq.heappop(evq)
        now = ev.time
        if now > cfg.T: break

        if ev.etype == EventType.ARRIVAL:
            controller.on_arrival(ev.payload["req"], now, evq)

        elif ev.etype == EventType.CTRL_FINISH:
            sid = ev.payload["sid"]
            req: Request = controller.on_finish(sid, now)
            pool.on_ctrl_finish(req, now, evq)
            controller.try_start_service(now, evq)

        elif ev.etype == EventType.RES_ARRIVAL:
            rname = ev.payload["res"]
            req: Request = ev.payload["req"]
            pool.on_res_arrival(rname, req, now, evq)

        elif ev.etype == EventType.RES_FINISH:
            rname = ev.payload["res"]
            pool.on_res_finish(rname, now, evq)

    # assemble results
    finished: List[Request] = [r for r in reqs if r.t_res_end is not None]

    def safe(x): return None if x is None else float(x)

    rows = []
    for r in finished:
        wait_ctrl = (r.t_ctrl_start - r.arrival_time) if (r.t_ctrl_start is not None) else None
        svc_ctrl  = (r.t_ctrl_end - r.t_ctrl_start) if (r.t_ctrl_end is not None and r.t_ctrl_start is not None) else None
        transfer  = (r.t_route_done - r.t_ctrl_end) if (r.t_route_done is not None and r.t_ctrl_end is not None) else None
        wait_res  = (r.t_res_start - r.t_route_done) if (r.t_res_start is not None and r.t_route_done is not None) else None
        svc_res   = (r.t_res_end - r.t_res_start) if (r.t_res_end is not None and r.t_res_start is not None) else None
        total     = (r.t_res_end - r.arrival_time) if (r.t_res_end is not None) else None

        rows.append({
            "rid": r.rid,
            "group": r.group,
            "priority": r.priority,
            "arrival": safe(r.arrival_time),
            "deadline": safe(r.deadline_time),
            "ctrl_start": safe(r.t_ctrl_start),
            "ctrl_end": safe(r.t_ctrl_end),
            "routed_to": r.routed_to,
            "route_done": safe(r.t_route_done),
            "res_start": safe(r.t_res_start),
            "res_end": safe(r.t_res_end),
            "wait_ctrl": safe(wait_ctrl),
            "svc_ctrl": safe(svc_ctrl),
            "transfer": safe(transfer),
            "wait_res": safe(wait_res),
            "svc_res": safe(svc_res),
            "total_response": safe(total),
        })

    return pd.DataFrame(rows)


In [11]:
def run_policy_sweep(cfg_base: SimConfig, group_mix: Dict[str, float],
                     policies: List[PolicyType]) -> pd.DataFrame:
    """
    Runs multiple controller policies on the same base config (cloned per run)
    and returns a compact summary table.
    """
    rows = []
    for pol in policies:
        cfg = SimConfig(
            chi_arrival=cfg_base.chi_arrival,
            lambda_ctrl=cfg_base.lambda_ctrl,
            lambda_res=dict(cfg_base.lambda_res),
            setup_delay=cfg_base.setup_delay,
            T=cfg_base.T,
            N_ctrl=cfg_base.N_ctrl,
            transfer_overhead=dict(cfg_base.transfer_overhead),
            route_probs=dict(cfg_base.route_probs),
            ctrl_policy=pol,
            group_priority=dict(cfg_base.group_priority),
            default_deadline_slots=dict(cfg_base.default_deadline_slots),
            seed=cfg_base.seed
        )
        df = run_simulation(cfg, group_mix)
        overall, per_group, miss = summarize_metrics(df)

        row = {
            "policy": pol.name,
            "count": overall.loc["total_response", "count/valid"],
            "total_mean": overall.loc["total_response", "mean"],
            "total_p95": overall.loc["total_response", "p95"],
            "wait_ctrl_mean": overall.loc["wait_ctrl", "mean"],
            "wait_res_mean": overall.loc["wait_res", "mean"],
            "svc_ctrl_mean": overall.loc["svc_ctrl", "mean"],
            "svc_res_mean": overall.loc["svc_res", "mean"],
            "transfer_mean": overall.loc["transfer", "mean"],
            "deadline_miss_rate": np.nan
        }
        if miss is not None and "__OVERALL__" in miss.index:
            row["deadline_miss_rate"] = float(miss.loc["__OVERALL__", "miss_rate"])

        rows.append(row)

    return pd.DataFrame(rows).sort_values(by=["total_mean", "total_p95"]).reset_index(drop=True)

def with_load(cfg: SimConfig, chi_new: float, seed: Optional[int] = None) -> SimConfig:
    return SimConfig(
        chi_arrival=chi_new,
        lambda_ctrl=cfg.lambda_ctrl,
        lambda_res=dict(cfg.lambda_res),
        setup_delay=cfg.setup_delay,
        T=cfg.T,
        N_ctrl=cfg.N_ctrl,
        transfer_overhead=dict(cfg.transfer_overhead),
        route_probs=dict(cfg.route_probs),
        ctrl_policy=cfg.ctrl_policy,
        group_priority=dict(cfg.group_priority),
        default_deadline_slots=dict(cfg.default_deadline_slots),
        seed=cfg.seed if seed is None else seed
    )


In [17]:
# === Utilization & deadline dashboards (re-add) ===
def utilization_report(cfg: SimConfig, df: pd.DataFrame) -> pd.DataFrame:
    """
    Rough utilization ρ estimates over horizon T:
      - Controller: sum(svc_ctrl) / (N_ctrl * T)
      - Resource s: sum(svc_res for routed_to==s) / T
    """
    if df.empty:
        return pd.DataFrame()

    out = []
    ctrl_busy = df["svc_ctrl"].dropna().sum()
    rho_ctrl = (ctrl_busy / cfg.N_ctrl) / cfg.T
    out.append({"unit": "CONTROLLER", "rho": rho_ctrl, "busy_time": ctrl_busy, "servers": cfg.N_ctrl})

    for rname in cfg.lambda_res.keys():
        busy = df.loc[df["routed_to"] == rname, "svc_res"].dropna().sum()
        rho = busy / cfg.T
        out.append({"unit": f"RES::{rname}", "rho": rho, "busy_time": busy, "servers": 1})

    rep = pd.DataFrame(out)
    rep["rho_clipped"] = rep["rho"].clip(upper=1.0)
    rep["warning"] = np.where(rep["rho"] >= 1.0, "UNSTABLE (ρ≥1)", "")
    return rep


def deadline_dashboard(df: pd.DataFrame) -> pd.DataFrame:
    """
    Per-group and overall deadline meeting stats.
    A request 'meets deadline' iff res_end <= deadline. NaN deadlines are ignored.
    """
    if "deadline" not in df.columns or df["deadline"].isna().all():
        return pd.DataFrame({"note": ["no deadlines present"]})

    m = df.dropna(subset=["deadline"]).copy()
    if m.empty:
        return pd.DataFrame({"note": ["no finite deadlines present"]})

    m["met"] = (m["res_end"] <= m["deadline"]).astype(int)
    g = m.groupby("group", dropna=False)["met"].agg(count="count", met="sum")
    g["misses"] = g["count"] - g["met"]
    g["miss_rate"] = g["misses"] / g["count"]

    overall = pd.DataFrame({
        "count": [g["count"].sum()],
        "met": [g["met"].sum()],
        "misses": [g["misses"].sum()],
        "miss_rate": [g["misses"].sum() / max(1, g["count"].sum())]
    }, index=["__OVERALL__"])

    return pd.concat([g, overall])


In [18]:
# # example usage
cfg = SimConfig(
    chi_arrival=8.0,
    # lambda_ctrl=10.0,
    # lambda_ctrl=6.0,
    lambda_ctrl=6.0,
    lambda_res={"PV":6.0, "BAT":12.0, "GRID":20.0},
    # setup_delay=0.02,
    setup_delay=0.05,
    T=240.0,
    # N_ctrl=2,
    N_ctrl=2,
    transfer_overhead={"PV":0.03, "BAT":0.02, "GRID":0.01},
    route_probs={"PV":0.35, "BAT":0.25, "GRID":0.40},
    ctrl_policy=PolicyType.FIFO,
)
group_mix = {"essential":0.2, "delay_sensitive":0.4, "delay_tolerant":0.4}

df = run_simulation(cfg, group_mix)
overall, per_group, miss = summarize_metrics(df)
rep = utilization_report(cfg, df)
dash = deadline_dashboard(df)

display(df.head(), overall, per_group, miss, rep, dash)

# try NPPS / EDF / WRR
table = run_policy_sweep(cfg, group_mix, [PolicyType.FIFO, PolicyType.NPPS, PolicyType.WRR, PolicyType.EDF])
display(table)

# stress
sweep_light = run_policy_sweep(with_load(cfg, 6.0), group_mix, [PolicyType.FIFO, PolicyType.NPPS, PolicyType.WRR, PolicyType.EDF])
sweep_heavy = run_policy_sweep(with_load(cfg, 12.0), group_mix, [PolicyType.FIFO, PolicyType.NPPS, PolicyType.WRR, PolicyType.EDF])
display(sweep_light, sweep_heavy)


,rid,group,priority,arrival,deadline,ctrl_start,ctrl_end,routed_to,route_done,res_start,res_end,wait_ctrl,svc_ctrl,transfer,wait_res,svc_res,total_response
0,0,delay_sensitive,2,0.300526,3.876320,0.300526,0.354521,GRID,0.364521,0.364521,0.421529,0.00000,0.053995,0.01,0.0,0.057008,0.121003
1,1,essential,3,0.335500,NaN,0.335500,0.466591,PV,0.496591,0.496591,0.593968,0.00000,0.131091,0.03,0.0,0.097377,0.258468
2,2,delay_tolerant,1,0.517083,10.591662,0.517083,0.735343,PV,0.765343,0.765343,0.845155,0.00000,0.218260,0.03,0.0,0.079812,0.328072
3,3,delay_sensitive,2,0.526995,2.639389,0.526995,0.765019,BAT,0.785019,0.785019,0.851655,0.00000,0.238024,0.02,0.0,0.066636,0.324660
4,4,delay_tolerant,1,0.663123,11.067977,0.735343,0.864634,PV,0.894634,0.894634,0.963757,0.07222,0.129291,0.03,0.0,0.069123,0.300635


,count/valid,mean,median,p95
svc_ctrl,1937.0,0.220269,0.168366,0.556891
svc_res,1937.0,0.147245,0.104889,0.388219
total_response,1937.0,1.153724,0.954623,2.771702
transfer,1937.0,0.019200,0.020000,0.030000
wait_ctrl,1937.0,0.687956,0.457092,2.237093
wait_res,1937.0,0.079053,0.000000,0.510342


count/valid      mean    median       p95
group                                                                    
delay_sensitive wait_ctrl             805.0  0.717037  0.477634  2.347823
                svc_ctrl              805.0  0.219975  0.175016  0.546921
                transfer              805.0  0.019056  0.020000  0.030000
                wait_res              805.0  0.078197  0.000000  0.478625
                svc_res               805.0  0.151047  0.106610  0.411684
                total_response        805.0  1.185312  0.974401  2.901088
delay_tolerant  wait_ctrl             748.0  0.658756  0.461610  2.119072
                svc_ctrl              748.0  0.218837  0.159923  0.578491
                transfer              748.0  0.019211  0.020000  0.030000
                wait_res              748.0  0.075593  0.000000  0.484590
                svc_res               748.0  0.147733  0.103668  0.384841
                total_response        748.0  1.120130  0.959200  2.592563
essential       wait_ctrl             384.0  0.683873  0.388563  2.447818
                svc_ctrl              384.0  0.223675  0.168001  0.553981
                transfer              384.0  0.019479  0.020000  0.030000
                wait_res              384.0  0.087587  0.000000  0.555238
                svc_res               384.0  0.138326  0.102942  0.336761
                total_response        384.0  1.152940  0.918011  2.842474

,count,met,misses,miss_rate
delay_sensitive,805,693,112,0.139130
delay_tolerant,748,747,1,0.001337
__OVERALL__,1553,1440,113,0.072762


,unit,rho,busy_time,servers,rho_clipped,warning
0,CONTROLLER,0.888878,426.661606,2,0.888878,
1,RES::PV,0.590177,141.642489,1,0.590177,
2,RES::BAT,0.268807,64.513576,1,0.268807,
3,RES::GRID,0.329409,79.058158,1,0.329409,


,count,met,misses,miss_rate
delay_sensitive,805,693,112,0.139130
delay_tolerant,748,747,1,0.001337
__OVERALL__,1553,1440,113,0.072762


,policy,count,total_mean,total_p95,wait_ctrl_mean,wait_res_mean,svc_ctrl_mean,svc_res_mean,transfer_mean,deadline_miss_rate
0,FIFO,1937.0,1.153724,2.771702,0.687956,0.079053,0.220269,0.147245,0.0192,0.072762
1,NPPS,1937.0,1.153724,4.066521,0.687956,0.079053,0.220269,0.147245,0.0192,0.037347
2,EDF,1937.0,1.153724,4.722028,0.687956,0.079053,0.220269,0.147245,0.0192,0.005795
3,WRR,1937.0,1.153724,3.531817,0.687956,0.079053,0.220269,0.147245,0.0192,0.054733


,policy,count,total_mean,total_p95,wait_ctrl_mean,wait_res_mean,svc_ctrl_mean,svc_res_mean,transfer_mean,deadline_miss_rate
0,FIFO,1450.0,0.519925,1.127047,0.090694,0.060776,0.205937,0.143326,0.019193,0.002577
1,WRR,1450.0,0.519925,1.182455,0.090694,0.060776,0.205937,0.143326,0.019193,0.003436
2,NPPS,1450.0,0.519925,1.189803,0.090694,0.060776,0.205937,0.143326,0.019193,0.004296
3,EDF,1450.0,0.519925,1.191631,0.090694,0.060776,0.205937,0.143326,0.019193,0.000859


,policy,count,total_mean,total_p95,wait_ctrl_mean,wait_res_mean,svc_ctrl_mean,svc_res_mean,transfer_mean,deadline_miss_rate
0,EDF,2179.0,9.185832,19.620614,8.687975,0.109561,0.219844,0.149058,0.019394,0.701632
1,NPPS,2179.0,12.326377,105.993812,11.828519,0.109561,0.219844,0.149058,0.019394,0.306301
2,WRR,2179.0,20.419861,96.222234,19.922004,0.109561,0.219844,0.149058,0.019394,0.830424
3,FIFO,2179.0,27.974791,56.892589,27.476933,0.109561,0.219844,0.149058,0.019394,0.898276


In [1]:
# Next steps 

# Resource-side scheduling (carry priority/deadlines into PV/BAT/GRID): add res_policy (FIFO/NPPS/EDF/WRR) so differences show up in wait_res under PV stress.

# Outage scenarios: inject on/off events for sources; measure impact on delays & miss rate.

# (then) Hybrid selector and ML tasks (forecasting & clustering) atop the stable baseline.